In [267]:
import itertools
import numpy as np
from sympy.utilities.iterables import multiset_permutations
from fractions import Fraction

In [268]:
votes = \
[
    [1, 0, 0, 1, 1, 0, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 1, 1, 1, 1],
    [1, 1, 0, 0, 0, 1, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 1],
]
k = 4

In [269]:
def calc_av(votes, k):
    approvals = np.array(votes).sum(axis=0)
    sorted_a = sorted([(v,i+1) for i,v in enumerate(approvals)], reverse=True)
    required = list()
    optional = list()
    while(True):
        max_score = sorted_a[0][0]
        maximums = [i for v,i in sorted_a if v == max_score]
        if k - len(required) >= len(maximums):
            required += maximums
            sorted_a = [(v,i) for v,i in sorted_a if v != max_score]
        else:
            optional = maximums
            break
    winners = []
    for perm in itertools.combinations(optional, k-len(required)):
        winners.append(sorted(required + list(perm)))
    return winners

calc_av(votes, k)

[[1, 6, 7, 8], [1, 4, 7, 8], [1, 4, 6, 7]]

In [270]:
def committees(votes, k):
    return multiset_permutations([1]*k + [0]*(len(votes[0])-k))

def bruteforce(votes, k, rule):
    max_score = 0
    for c in committees(votes, k):
        max_score = max(max_score, rule(c, votes, k))
    winners = []
    for c in committees(votes, k):
        if rule(c, votes, k) == max_score:
            winners.append(c)
    return winners

def candidates(committee):
    return [i+1 for i,v in enumerate(committee) if v == 1]

def prettify(winners):
    return str([["c_"+str(i) for i in candidates(winner)] for winner in winners]).replace("[", "\\{").replace("]", "\\}").replace("'", "")

In [271]:
def av(committee, votes, k):
    return sum(np.array(votes).sum(axis=0) * (np.array(committee)))

def cc(committee, votes, k):
    return sum([(np.array(vote) * np.array(committee)).any() for vote in votes])

def pav(committee, votes, k):
    approvals = np.array(votes).sum(axis=0)
    harmonics = np.array([float(sum(Fraction(1, i) for i in range(1, x+1))) for x in approvals])
    return sum(harmonics * (np.array(committee)))

In [272]:
av_winners = bruteforce(votes, k, av)
cc_winners = bruteforce(votes, k, cc)
pav_winners = bruteforce(votes, k, pav)

In [273]:
print("Winners under AV: ", prettify(av_winners))
print("Winners under AV: ", prettify(cc_winners))
print("Winners under AV: ", prettify(pav_winners))

Winners under AV:  \{\{c_1, c_6, c_7, c_8\}, \{c_1, c_4, c_7, c_8\}, \{c_1, c_4, c_6, c_7\}\}
Winners under AV:  \{\{c_6, c_7, c_8, c_9\}, \{c_5, c_7, c_8, c_9\}, \{c_5, c_6, c_7, c_9\}, \{c_4, c_7, c_8, c_9\}, \{c_4, c_6, c_8, c_9\}, \{c_4, c_6, c_7, c_9\}, \{c_4, c_5, c_8, c_9\}, \{c_4, c_5, c_6, c_9\}, \{c_3, c_7, c_8, c_9\}, \{c_3, c_6, c_7, c_9\}, \{c_3, c_5, c_8, c_9\}, \{c_3, c_5, c_6, c_9\}, \{c_3, c_4, c_8, c_9\}, \{c_3, c_4, c_6, c_9\}, \{c_2, c_7, c_8, c_9\}, \{c_2, c_6, c_7, c_9\}, \{c_2, c_5, c_8, c_9\}, \{c_2, c_5, c_7, c_9\}, \{c_2, c_5, c_6, c_9\}, \{c_2, c_4, c_8, c_9\}, \{c_2, c_4, c_7, c_9\}, \{c_2, c_4, c_6, c_9\}, \{c_2, c_3, c_7, c_9\}, \{c_2, c_3, c_5, c_9\}, \{c_2, c_3, c_4, c_9\}, \{c_1, c_7, c_8, c_9\}, \{c_1, c_6, c_8, c_9\}, \{c_1, c_6, c_7, c_9\}, \{c_1, c_6, c_7, c_8\}, \{c_1, c_5, c_8, c_9\}, \{c_1, c_5, c_7, c_9\}, \{c_1, c_5, c_7, c_8\}, \{c_1, c_5, c_6, c_9\}, \{c_1, c_5, c_6, c_8\}, \{c_1, c_5, c_6, c_7\}, \{c_1, c_4, c_8, c_9\}, \{c_1, c_4, c_7, c_9\

In [274]:
def check_satisfied_l(W, V, k, l):
    # for all groupsizes
    for g in range(int(l * len(V)/k + 0.5), len(V)+1):
        # for all subgroups of this size
        for mask in multiset_permutations([True] * g + [False] * (len(V) - g)):
            # is l-cohesive?
            cohesiveness = np.prod(np.vstack(np.array(V)[mask]), axis=0).sum()
            if cohesiveness < l:
            # if not, don't care
                continue
            # is l-satisfied?
            approvals = np.array(V)[mask].sum(axis=0) * np.array(W)
            satisfication = np.count_nonzero(approvals)
            if satisfication < l:
                return False
    return True

def check_jr(W, V, k):
    return check_satisfied_l(W, V, k, 1)

def check_ejr(W, V, k):
    satisfies_l = [check_satisfied_l(W, V, k, l) for l in range(1, k+1)]
    if False in satisfies_l:
        return False
    return True

def check_cs(W, V, k):
    for l in range(1, k+1):
        for g in range(int(l * len(V)/k + 0.5), len(V)+1):
            for S in multiset_permutations([True] * g + [False] * (len(V) - g)):
                for U in multiset_permutations([1] * l + [0] * (len(V[0]) - l)):
                    if ((np.array(V)[S] * U).sum(axis=1) - (np.array(V)[S] * W).sum(axis=1)).min() > 0:
                        return False, S, U #f"S={candidates(S)} prefer U={candidates(U)} by {((np.array(V)[S] * U).sum(axis=1) - (np.array(V)[S] * W).sum(axis=1))}"
    return True, None, None

In [275]:
for committee in av_winners:
    print("Candidates:", candidates(committee), "Satisfies JR:", check_jr(committee, votes, k), "Satisfies EJR:",  check_ejr(committee, votes, k), "Satisfies CS:", check_cs(committee, votes, k))
for committee in cc_winners:
    print("Candidates:", candidates(committee), "Satisfies JR:", check_jr(committee, votes, k), "Satisfies EJR:",  check_ejr(committee, votes, k), "Satisfies CS:", check_cs(committee, votes, k))
for committee in pav_winners:
    print("Candidates:", candidates(committee), "Satisfies JR:", check_jr(committee, votes, k), "Satisfies EJR:",  check_ejr(committee, votes, k), "Satisfies CS:", check_cs(committee, votes, k))

Candidates: [1, 6, 7, 8] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [1, 4, 7, 8] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [1, 4, 6, 7] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [6, 7, 8, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [5, 7, 8, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [5, 6, 7, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [4, 7, 8, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [4, 6, 8, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [4, 6, 7, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [4, 5, 8, 9] Satisfies JR: True Satisfies EJR: True Satisfies CS: (True, None, None)
Candidates: [4, 5, 6, 9] Satis

In [276]:
for committee in committees(votes, k):
    if committee not in av_winners + cc_winners + pav_winners:
        continue
    print("$"+str(["c_"+str(i) for i in candidates(committee)]).replace("'", "").replace("[", "\{").replace("]","\}")+"$", end="&")
    print(committee in av_winners, end="&")
    print(committee in cc_winners, end="&")
    print(committee in pav_winners, end="&")
    print(check_jr(committee, votes, k), end="&")
    print(check_ejr(committee, votes, k), end="&")
    CS, S, U = check_cs(committee, votes, k)
    print(CS, end="")
    if not CS:
        print(". $S="+str(["v_"+str(i) for i in candidates(S)]).replace("'", "").replace("[", "\{").replace("]","\}")+"$",end="")
        print(" prefer $U="+str(["c_"+str(i) for i in candidates(U)]).replace("'", "").replace("[", "\{").replace("]","\}")+"$",end="")
    print(end="\\\\\n")

$\{c_6, c_7, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_5, c_7, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_5, c_6, c_7, c_9\}$&False&True&False&True&True&True\\
$\{c_4, c_7, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_4, c_6, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_4, c_6, c_7, c_9\}$&False&True&False&True&True&True\\
$\{c_4, c_5, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_4, c_5, c_6, c_9\}$&False&True&False&True&True&True\\
$\{c_3, c_7, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_3, c_6, c_7, c_9\}$&False&True&False&True&True&True\\
$\{c_3, c_5, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_3, c_5, c_6, c_9\}$&False&True&False&True&True&True\\
$\{c_3, c_4, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_3, c_4, c_6, c_9\}$&False&True&False&True&True&True\\
$\{c_2, c_7, c_8, c_9\}$&False&True&False&True&True&True\\
$\{c_2, c_6, c_7, c_9\}$&False&True&False&True&True&True\\
$\{c_2, c_5, c_8, c_9\}$&False&True&False&True&True&True